In [3]:
import csv
import pandas as pd
import os
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

############################################################################################
#Initial variables
#This part of the code is used to set the inital parameters for the web crawler
############################################################################################
START_PAGE = 51
END_PAGE = 125
PROJECT_ABBR = "usnews"
PROJECT_NAME = "Project 5"
URL = "https://www.usnews.com/education/best-global-universities/rankings"

#Main window
driver =webdriver.Firefox(executable_path=r'D:\git\FINA2390-Programs\geckodriver.exe')
driver.get(URL)

#Secondary window
driver2 = webdriver.Firefox(executable_path=r'D:\git\FINA2390-Programs\geckodriver.exe')

df_columns = ['Ranking',
              'University Name',
              'Country',
              'Address',
              'Website',
              'No. of Students',
              'Source']

df_etf = pd.DataFrame(columns=df_columns)

############################################################################################
#Class
#This particular class to define the conditions for explict wait
#The condition states that the crawler will jump to next page if and only if the webpage can find the text presented
############################################################################################
class text_appear(object):
  def __init__(self, locator1, locator2, locator3):
    self.locator1 = locator1
    self.locator2 = locator2
    self.locator3 = locator3

  def __call__(self, driver):
    element1 = driver.find_element(*self.locator1) 
    element2 = driver.find_element(*self.locator2)
    element3 = driver.find_element(*self.locator3)
    if len(element1.text) != 0 and len(element2.text) !=0 and len(element3.text) !=0:
        return True
    else:
        return False

def go_to_page(page):
    if page != 1:
        print 'GOING TO PAGE' + str(page)
        driver.get('https://www.usnews.com/education/best-global-universities/rankings?page='+str(page))
        print 'ARRIVED PAGE' + str(page)
    else:
        return

def save_file(state, CURRENT_PAGE):
    if state == 'backup':
        filename = '['+ PROJECT_ABBR +'][Intrim Backup][' + str(CURRENT_PAGE) + ']' + PROJECT_NAME + ' PAGE ' + str(START_PAGE) + ' to PAGE ' + str(CURRENT_PAGE) + ' Records.csv'
        df_etf.to_csv(filename, index=False,  encoding='utf-8')
        print '[Intrim Backup]Saved First '+ str(CURRENT_PAGE) + 'Pages Records to CSV'
    elif state == 'all':
        filename = '['+ PROJECT_ABBR +']'+ PROJECT_NAME +' PAGE ' + str(START_PAGE) + 'to PAGE ' + str(END_PAGE) + 'Records.csv'
        df_etf.to_csv(filename, index=False, encoding='utf-8')
        print 'Exported to csv'
        
    
def dig(link, row_index):
    
    #Open a new window
    driver2.get(link.get_attribute('href'))
    time.sleep(1)
    try:    
        element = WebDriverWait(driver2, 5).until(           
            text_appear((By.XPATH, '//*[@id="directoryPageSection-institution-data"]/div[1]/div[1]'), (By.XPATH, '//*[@id="content"]/div[3]/div[1]/div[1]/div[4]/div[2]/a'),(By.XPATH, '//*[@id="content"]/div[3]/div[1]/div[1]/div[3]/div[2]/div[1]')))
    except TimeoutException as ex:
        return False
    except StaleElementReferenceException as sx:
        return False
    finally:
        #find fields on the new driver
        time.sleep(1)
        if len(driver2.find_elements_by_xpath('//*[@id="directoryPageSection-institution-data"]/div[1]/div[1]')) != 0:
            df_etf.loc[row_index, 'No. of Students'] = driver2.find_element_by_xpath('//*[@id="directoryPageSection-institution-data"]/div[1]/div[1]').text
        if len(driver2.find_elements_by_xpath('//*[@id="content"]/div[3]/div[1]/div[1]/div[4]/div[2]/a')) != 0:
            df_etf.loc[row_index, 'Website']         = driver2.find_element_by_xpath('//*[@id="content"]/div[3]/div[1]/div[1]/div[4]/div[2]/a').text
        if len(driver2.find_elements_by_xpath('//*[@id="content"]/div[3]/div[1]/div[1]/div[3]/div[2]/div[1]')) != 0:    
            df_etf.loc[row_index, 'Address']         = driver2.find_element_by_xpath('//*[@id="content"]/div[3]/div[1]/div[1]/div[3]/div[2]/div[1]').text
############################################################################################
#Main
#This part is the main function of the program
############################################################################################
def main():
    time.sleep(2)
    
    go_to_page(START_PAGE)
    time.sleep(1)
    print 'Start at PAGE'+ str(START_PAGE)
    
    n = 1
    for CURRENT_PAGE in range(START_PAGE,END_PAGE+1):
        print '--------------------PAGE ' + str(CURRENT_PAGE) +' START--------------------'
        START_TIMESTAMP = time.time()
        for row in driver.find_elements_by_xpath('//*[@id="resultsMain"]/div[1]/div'):
            df_etf.loc[n, 'Ranking']         = row.find_element_by_xpath('div[2]/span').text
            df_etf.loc[n, 'University Name'] = row.find_element_by_xpath('div[3]/h2/a').text
            df_etf.loc[n, 'Country']         = row.find_element_by_xpath('div[3]/div/span[1]').text
            link = row.find_element_by_xpath('div[3]/h2/a')
            
            #open the univesity profile to get other fields
            dig(link, n)
            print '--------------------ROW ' + str(n)+ ' FINISHED--------------------'
            n = n + 1
        print '--------------------PAGE ' + str(CURRENT_PAGE) +' FINISH--------------------'
        END_TIMESTAMP = time.time()
        print '-TIME USED:' + str(START_TIMESTAMP-END_TIMESTAMP) + '-'
        
        #store every first 5,10,15... pages to CSV for backup purpose
        if(CURRENT_PAGE % 5) == 0:
            save_file('backup', CURRENT_PAGE)
            
        go_to_page(CURRENT_PAGE + 1)
        
    #set Source as QS for every row as this is crawl from QS
    df_etf['Source'] = PROJECT_ABBR
    
    #export the file to CSV
    save_file('all', CURRENT_PAGE)
    
    #close the driver
    driver.quit()
if __name__ == "__main__":
    main()



GOING TO PAGE51
ARRIVED PAGE51
Start at PAGE51
--------------------PAGE 51 START--------------------
--------------------ROW 1 FINISHED--------------------
--------------------ROW 2 FINISHED--------------------
--------------------ROW 3 FINISHED--------------------
--------------------ROW 4 FINISHED--------------------
--------------------ROW 5 FINISHED--------------------
--------------------ROW 6 FINISHED--------------------
--------------------ROW 7 FINISHED--------------------
--------------------ROW 8 FINISHED--------------------
--------------------ROW 9 FINISHED--------------------
--------------------ROW 10 FINISHED--------------------
--------------------PAGE 51 FINISH--------------------
-TIME USED:-139.090000153-
GOING TO PAGE52
ARRIVED PAGE52
--------------------PAGE 52 START--------------------
--------------------ROW 11 FINISHED--------------------
--------------------ROW 12 FINISHED--------------------
--------------------ROW 13 FINISHED--------------------
-------------

--------------------ROW 111 FINISHED--------------------
--------------------ROW 112 FINISHED--------------------
--------------------ROW 113 FINISHED--------------------
--------------------ROW 114 FINISHED--------------------
--------------------ROW 115 FINISHED--------------------
--------------------ROW 116 FINISHED--------------------
--------------------ROW 117 FINISHED--------------------
--------------------ROW 118 FINISHED--------------------
--------------------ROW 119 FINISHED--------------------
--------------------ROW 120 FINISHED--------------------
--------------------PAGE 62 FINISH--------------------
-TIME USED:-140.487999916-
GOING TO PAGE63
ARRIVED PAGE63
--------------------PAGE 63 START--------------------
--------------------ROW 121 FINISHED--------------------
--------------------ROW 122 FINISHED--------------------
--------------------ROW 123 FINISHED--------------------
--------------------ROW 124 FINISHED--------------------
--------------------ROW 125 FINISHE

--------------------ROW 221 FINISHED--------------------
--------------------ROW 222 FINISHED--------------------
--------------------ROW 223 FINISHED--------------------
--------------------ROW 224 FINISHED--------------------
--------------------ROW 225 FINISHED--------------------
--------------------ROW 226 FINISHED--------------------
--------------------ROW 227 FINISHED--------------------
--------------------ROW 228 FINISHED--------------------
--------------------ROW 229 FINISHED--------------------
--------------------ROW 230 FINISHED--------------------
--------------------PAGE 73 FINISH--------------------
-TIME USED:-122.335999966-
GOING TO PAGE74
ARRIVED PAGE74
--------------------PAGE 74 START--------------------
--------------------ROW 231 FINISHED--------------------
--------------------ROW 232 FINISHED--------------------
--------------------ROW 233 FINISHED--------------------
--------------------ROW 234 FINISHED--------------------
--------------------ROW 235 FINISHE

--------------------ROW 331 FINISHED--------------------
--------------------ROW 332 FINISHED--------------------
--------------------ROW 333 FINISHED--------------------
--------------------ROW 334 FINISHED--------------------
--------------------ROW 335 FINISHED--------------------
--------------------ROW 336 FINISHED--------------------
--------------------ROW 337 FINISHED--------------------
--------------------ROW 338 FINISHED--------------------
--------------------ROW 339 FINISHED--------------------
--------------------ROW 340 FINISHED--------------------
--------------------PAGE 84 FINISH--------------------
-TIME USED:-117.289000034-
GOING TO PAGE85
ARRIVED PAGE85
--------------------PAGE 85 START--------------------
--------------------ROW 341 FINISHED--------------------
--------------------ROW 342 FINISHED--------------------
--------------------ROW 343 FINISHED--------------------
--------------------ROW 344 FINISHED--------------------
--------------------ROW 345 FINISHE

--------------------ROW 441 FINISHED--------------------
--------------------ROW 442 FINISHED--------------------
--------------------ROW 443 FINISHED--------------------
--------------------ROW 444 FINISHED--------------------
--------------------ROW 445 FINISHED--------------------
--------------------ROW 446 FINISHED--------------------
--------------------ROW 447 FINISHED--------------------
--------------------ROW 448 FINISHED--------------------
--------------------ROW 449 FINISHED--------------------
--------------------ROW 450 FINISHED--------------------
--------------------PAGE 95 FINISH--------------------
-TIME USED:-119.450000048-
[Intrim Backup]Saved First 95Pages Records to CSV
GOING TO PAGE96
ARRIVED PAGE96
--------------------PAGE 96 START--------------------
--------------------ROW 451 FINISHED--------------------
--------------------ROW 452 FINISHED--------------------
--------------------ROW 453 FINISHED--------------------
--------------------ROW 454 FINISHED------

ARRIVED PAGE106
--------------------PAGE 106 START--------------------
--------------------ROW 551 FINISHED--------------------
--------------------ROW 552 FINISHED--------------------
--------------------ROW 553 FINISHED--------------------
--------------------ROW 554 FINISHED--------------------
--------------------ROW 555 FINISHED--------------------
--------------------ROW 556 FINISHED--------------------
--------------------ROW 557 FINISHED--------------------
--------------------ROW 558 FINISHED--------------------
--------------------ROW 559 FINISHED--------------------
--------------------ROW 560 FINISHED--------------------
--------------------PAGE 106 FINISH--------------------
-TIME USED:-113.307999849-
GOING TO PAGE107
ARRIVED PAGE107
--------------------PAGE 107 START--------------------
--------------------ROW 561 FINISHED--------------------
--------------------ROW 562 FINISHED--------------------
--------------------ROW 563 FINISHED--------------------
-----------------

ARRIVED PAGE117
--------------------PAGE 117 START--------------------
--------------------ROW 661 FINISHED--------------------
--------------------ROW 662 FINISHED--------------------
--------------------ROW 663 FINISHED--------------------
--------------------ROW 664 FINISHED--------------------
--------------------ROW 665 FINISHED--------------------
--------------------ROW 666 FINISHED--------------------
--------------------ROW 667 FINISHED--------------------
--------------------ROW 668 FINISHED--------------------
--------------------ROW 669 FINISHED--------------------
--------------------ROW 670 FINISHED--------------------
--------------------PAGE 117 FINISH--------------------
-TIME USED:-115.450000048-
GOING TO PAGE118
ARRIVED PAGE118
--------------------PAGE 118 START--------------------
--------------------ROW 671 FINISHED--------------------
--------------------ROW 672 FINISHED--------------------
--------------------ROW 673 FINISHED--------------------
-----------------

In [33]:
df_etf

,Ranking,University Name,Country,Address,Website,No. of Students,Source
1,#1,Harvard University,United States,"Cambridge, Massachusetts 02138",www.harvard.edu,"20,326",usnews
2,#2,Massachusetts Institute of Technology,United States,77 Massachusetts Avenue,web.mit.edu,"11,177",usnews
3,#3,Stanford University,United States,"Stanford, California 94305",www.stanford.edu,"15,845",usnews
4,#4,University of California--Berkeley,United States,110 Sproul Hall,berkeley.edu,"38,204",usnews
5,#5,University of Oxford,United Kingdom,NaN,NaN,"20,256",usnews
6,#6,California Institute of Technology,United States,1200 E. California Boulevard,www.caltech.edu,"2,209",usnews
7,#7,University of Cambridge,United Kingdom,NaN,NaN,"18,875",usnews
8,#8,Columbia University,United States,2960 Broadway,www.columbia.edu,"26,587",usnews
9,#9,Princeton University,United States,"Princeton, New Jersey 08544",www.princeton.edu,"7,955",usnews
10,#10\nTie,Johns Hopkins University,United States,3400 N. Charles Street,www.jhu.edu,"15,498",usnews
